# Using FlexBe with ROS

<img src="img/flexbe_img.png" width="1000" />

<img src="img/robotignite_logo_text.png" width="400"/>

## Unit 1: Create a basic behavior

<p style="background:green;color:white;">SUMMARY</p>

Estimated time to completion: <b>2 hours</b><br><br>
In this unit, you are going to learn the basic concepts of FlexBe, and you are going to review, step by step, how to create a basic behavior and a basic state.

<p style="background:green;color:white;">END OF SUMMARY</p>

Before beginning work with FlexBe, let's have a look at some basic concepts you need to know.

FlexBe allows you to create complex robot behaviors without the need of manually coding them. Based on basic capabilities, which interface standard functionality, you can easily compose state machines using the drag and drop editor that FlexBe provides in its GUI. These basic capabilities are also known as **states**.

States define what should be done by a behavior. They are grouped as state machines, defining the control flow when the behavior is executed. Therefore, each state declares a set of outcomes, which represent possible results of execution. Furthermore, data can be shared and modified during runtime between states. This data is called userdata and is stored as key-value-pairs. Each state declares which keys it requires (input) and provides (output).

**FlexBE states** are the high-level building blocks from which behaviors are constructed and are supposed to interface with the capabilities of your robot system.

Typically, implementing states is the only time when you will manually write code while working with FlexBE. In the following exercise, you will create your first FlexBe state.

## Creating a new state

<p style="background:#EE9023;color:white;">**Exercise 1.1**</p>
<br>

a) Go to your **catkin_ws/src** folder and create a new FlexBe environment. You can do it by issuing the following commands.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd caktin_ws/src

In [ ]:
rosrun flexbe_widget create_repo turtlebot

If everything went OK, a new structure of folders and files will have appeared in your workspace, under the name of **turtlebot_behaviors**.

<img src="img/turtlebot_behavior_structure.png" width="400" />

b) Within this **turtlebot_behaviors** folder, you have all the structure needed to create a new FlexBe behavior, alongside some example files. If you navigate into **"turtlebot_behaviors/turtlebot_flexbe_states/src/turtlebot_flexbe_states"**, you will find a couple of example scripts for FlexBe states. You can have a look at them, if you wish, since they are very well commented. Inside this folder is where we will place our new code.

c) Whenever you create a new behavior, it will be necessary to compile your workspace. So, let's do that now.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd ..

In [ ]:
catkin_make

In [ ]:
source devel/setup.bash

d) Let's add our own state. For that, let's create a new script inside **"turtlebot_behaviors/turtlebot_flexbe_states/src/turtlebot_flexbe_states"**, which we can call **drive_forward.py**.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd src/turtlebot_behaviors/turtlebot_flexbe_states/src/turtlebot_flexbe_states/

In [ ]:
touch drive_forward.py;chmod +x drive_forward.py

Inside this new script, copy the following content.

<p style="background:green;color:white;">**drive_forward.py**</p>

In [ ]:
#!/usr/bin/env python
import rospy
from flexbe_core import EventState, Logger
from flexbe_core.proxy import ProxyPublisher, ProxySubscriberCached
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan

class GoFowardState(EventState):
    '''
	Driving state for a ground robot. This state allows the robot to drive forward a certain distance
    at a specified velocity/ speed.

	-- speed 	float 	Speed at which to drive the robot
    -- travel_dist float   How far to drive the robot before leaving this state
    -- obstacle_dist float Distance at which to determine blockage of robot path

	<= failed 			    If behavior is unable to ready on time
	<= done 				Example for a failure outcome.

	'''
    def __init__(self, speed, travel_dist, obstacle_dist):
        super(GoFowardState, self).__init__(outcomes=['failed', 'done'])
        self._start_time = None
        self.data = None
        self._speed = speed
        self._travel_dist = travel_dist
        self._obstacle_dist = obstacle_dist

        self.vel_topic = '/cmd_vel'
        self.scan_topic = '/kobuki/laser/scan'

        #create publisher passing it the vel_topic_name and msg_type
        self.pub = ProxyPublisher({self.vel_topic: Twist})
        #create subscriber
        self.scan_sub = ProxySubscriberCached({self.scan_topic: LaserScan})
        self.scan_sub.set_callback(self.scan_topic, self.scan_callback)
    
    def execute(self, userdata):
        if not self.cmd_pub:
            return 'failed'
        #run obstacle checks [index 0: left, 360: middle, 719: right]
        if(self.data is not None):
            Logger.loginfo('FWD obstacle distance is: %s' % self.data.ranges[360])
            if self.data.ranges[360] <= self._obstacle_dist:
                return 'failed'
    
            #measure distance travelled
            elapsed_time = (rospy.Time.now() - self._start_time).to_sec()
            distance_travelled = (elapsed_time/ 60.0) * self._speed
    
            if distance_travelled >= self._travel_dist:
                return 'done'
        
        #drive
        self.pub.publish(self.vel_topic, self.cmd_pub)

    def on_enter(self, userdata):
        Logger.loginfo("Drive FWD STARTED!")
        #set robot speed here
        self.cmd_pub = Twist()
        self.cmd_pub.linear.x = self._speed
        self.cmd_pub.angular.z = 0.0
        
    def on_exit(self, userdata):
        self.cmd_pub.linear.x = 0.0
        self.pub.publish(self.vel_topic, self.cmd_pub)
        self.scan_sub.unsubscribe_topic(self.scan_topic)
        Logger.loginfo("Drive FWD ENDED!")
        
    def on_start(self):
        Logger.loginfo("Drive FWD READY!")
        self._start_time = rospy.Time.now() #bug detected! (move to on_enter)
        
    def on_stop(self):
		Logger.loginfo("Drive FWD STOPPED!")

    def scan_callback(self, data):
        self.data = data

<p style="background:green;color:white;">**drive_forward.py**</p>

Don't worry if you don't understand the whole code right now. We'll review and explain everything in detail in this chapter. For now, all you need to know is that the purpose of the above state is to allow a robot to drive forward a certain distance at a specified velocity/speed.

e) Compile again your workspace.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roscd;cd ..

In [ ]:
catkin_make

In [ ]:
source devel/setup.bash

<p style="background:#EE9023;color:white;">**End of Exercise 1.1**</p>

So, basically, a state defines a basic functionality. In FlexBe, we call any class with the following form a state:

In [ ]:
class ClassNameState(EventState):
    def __init__(self):
        super(ClassNameState, self).__init__(outcomes=['done', 'fail'])
        
    def execute(self, userdata):
        pass

Now, from the code snippet above, we can tell three main things from FlexBe states:

- They inherit from the **EventState** class.
- They return an outcome in the form of: **done** or **fail**.
- They implement the **execute()** function.

But, as you may have noticed from the script you saw in Exercise 1.1, there are some more things involved. And you are totally right! We also have the state **events**. Currently, six implementable events exist: 

- **on_start**: This event is triggered when the **behavior** is started. *It is recommended, if possible, to initialize used resources in the constructor.*


- **on_enter**: This event is triggered when a **state** becomes active, i.e., when a transition occurs. *Primarily used to start actions that are associated with this state or initialize variables that may have changed during a previous execution.*


- **on_exit**: This event is triggered when an **outcome** is returned and another state becomes active. *It can be used to stop running processes that were started by on_enter.*


- **on_stop**: This event is triggered whenever the **behavior** stops execution, and also if it is cancelled. *Used for cleaning up claimed resources or running processes even if the state is not active.*


- **on_pause** (similar to on_exit): This event is triggered when:
    - the operator clicks on the Pause button in the user interface 
    or 
    - execution is paused automatically, e.g., because of a priority container becoming active. *It is recommended to implement this event similar to on_exit.*


- **on_resume**  (similar to on_enter): Complement to on_pause, i.e., this event is triggered when execution resumes after having been paused. *It is recommended to implement this event similar to on_enter.*

With the proper concepts presented, let's have a look at the following image.

<img src="img/state_lifecycle.png" width="800" />

The above image shows the lifecycle of a state, along with associated events. When a new behavior is executed, this lifecycle is started for all states contained anywhere in the hierarchy of the behavior. Basically, the process goes as follows:

1. First, all states are instantiated and thus, their constructor is called (**\__init __()**). 

2. Afterwards, i.e., when the execution of the constructor of all states has finished, the **on_start** event is triggered for all states. 

3. Then, the initial state becomes active. Whenever a state becomes active, its **on_enter** event is triggered once. 

4. Then, it periodically (default: 10 hz) calls its **execute** function to check for exit conditions. 

5. If an outcome is returned, the **on_exit** event is triggered once before the next state becomes active. 

6. Finally, if an outcome of the root-level state machine is returned or execution of the behavior is stopped manually, the **on_stop** event of all states is triggered in order to clean up.

Note that **on_start** and **on_stop** are triggered exactly once for each state (concurrently) per behavior execution. Although **on_enter** and **on_exit** are also only triggered once when the associated state becomes active or inactive, a state can become active multiple times and, thus, these events can be triggered multiple times during a behavior execution.

So now that we already know how the whole structure and lifecycle of a state works, let's have a closer look at the state that we created in the previous exercise, 1.1.

In [ ]:
#!/usr/bin/env python
import rospy
from flexbe_core import EventState, Logger
from flexbe_core.proxy import ProxyPublisher, ProxySubscriberCached
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan

First of all, we can see some imports that we will need for this class. From those imports, we can highlight the **EventState** and **Logger** classes, from the **flexbe_core** module. The EventState class is the basic state, from where all other states will inherit. The Logger class provides the FlexBe logging functionality.

Also, pay atenttion to the **ProxyPublisher** and **ProxySubscriberCached** classes, from **flexbe_core.proxy**. FlexBe provides a set of **proxies**, which are more efficient implementations of basic ROS elements, like Publishers or Subscribers. We'll see more on them right now.

In [ ]:
def __init__(self, speed, travel_dist, obstacle_dist):
    super(GoFowardState, self).__init__(outcomes=['failed', 'done'])
    self._start_time = None
    self.data = None
    self._speed = speed
    self._travel_dist = travel_dist
    self._obstacle_dist = obstacle_dist

    self.vel_topic = '/cmd_vel'
    self.scan_topic = '/kobuki/laser/scan'

    #create publisher passing it the vel_topic_name and msg_type
    self.pub = ProxyPublisher({self.vel_topic: Twist})
    #create subscriber
    self.scan_sub = ProxySubscriberCached({self.scan_topic: LaserScan})
    self.scan_sub.set_callback(self.scan_topic, self.scan_callback)

Next, we have the constructor of the class. As you can see, it takes as input of three parameters: **speed**, **travel_dist**, and **obstacle_dist**. These are the speed we want our robot to move at, the distance we want our robot to travel, and the maximum distance to an obstacle before the robot stops (to avoid a potential crash).

Next, we are initializing some variables we'll use on the class, including the inputs we've just talked about, which we also store in variables of the class.

Then, we are storing the topics we are going to use in variables. In the **/cmd_vel** topic, we will publish in order to move the robot. And we are going to subscribe to the **/kobuki/laser/scan** topic to get the readings from the laser mounted on the robot (in order to get the distance to obstacles). It is always recommended to store the topics in variables, since you will need them again later.

Finally, we define our proxies for publishing and for subscribing to the topics. So, we are using the **ProxyPublisher** and the **ProxySubscriberCached**. Four types of proxies exist in FlexBe:

* **ProxyPublisher**: It is the equivalent to the ROS Publisher.
* **ProxySubscriberCached**: It is the equivalent to the ROS Subscribers.
* **ProxyServiceCaller**: It is the equivalent to the ROS Service Client.
* **ProxyActionClient**: It is the equivalent to the ROS Action Client.

All the proxies are instantiated by passing them a dictionary of topics and their types to work with. For instance:

In [ ]:
self.pub = ProxyPublisher({'/a_topic': MsgType})

And you can pass as many topics as you want to it, just like this:

In [ ]:
self.pub = ProxyPublisher({'/a_topic': MsgType, '/another_topic': MsgType, ...})

Finally, we are also setting the callback function for the laser subscriber, using the **set_callback()** function.

In [ ]:
def execute(self, userdata):
        if not self.cmd_pub:
            return 'failed'
        #run obstacle checks [index 0: left, 360: middle, 719: right]
        if(self.data is not None):
            Logger.loginfo('FWD obstacle distance is: %s' % self.data.ranges[360])
            if self.data.ranges[360] <= self._obstacle_dist:
                return 'failed'
    
            #measure distance travelled
            elapsed_time = (rospy.Time.now() - self._start_time).to_sec()
            distance_travelled = (elapsed_time/ 60.0) * self._speed
    
            if distance_travelled >= self._travel_dist:
                return 'done'
        
        #drive
        self.pub.publish(self.vel_topic, self.cmd_pub)

Next we have the **execute()** function. Inside this function, we are checking several things. The workflow goes as follows:

1. First, we check if a message (in **self.cmd_pub**) to publish into the **/cmd_vel** topic exists. If not, then we set the state as **failed** and we exit.
2. Second, we check the data related to the laser readings (which is stored in **self.data**). If the readings indicate that there is an obstacle closer to the robot than the **self._obstacle_dist**, we exit and set the state to **failed**. Also, we check if the current distance that the robot has traveled is higher or equal to **self._travel_dist**, and if it is, we exit and set the state to **done**.
3. Finally, we publish into the **/cmd_vel** topic in order to move the robot.

Summarizing, we poll the scan data to check for obstacles in the path. If there are any, we return an outcome of **failed**, if not, we keep driving forward the specified distance, and return an outcome of **done** when finished.

In [ ]:
def on_enter(self, userdata):
    Logger.loginfo("Drive FWD STARTED!")
    #set robot speed here
    self.cmd_pub = Twist()
    self.cmd_pub.linear.x = self._speed
    self.cmd_pub.angular.z = 0.0

Next, we have the **on_enter()** state. This event is called when this state is transitioned to. Here, we are just creating the message to be published by the *ProxyPublisher.*

In [ ]:
def on_exit(self, userdata):
    self.cmd_pub.linear.x = 0.0
    self.pub.publish(self.vel_topic, self.cmd_pub)
    self.scan_sub.unsubscribe_topic(self.scan_topic)
    Logger.loginfo("Drive FWD ENDED!")

The **on_exit()** event is called when leaving this state. Here is where we stop the robot from driving any further. 

In [ ]:
def on_start(self):
    Logger.loginfo("Drive FWD READY!")
    self._start_time = rospy.Time.now() #bug detected! (move to on_enter)
    
def on_stop(self):
    Logger.loginfo("Drive FWD STOPPED!")

In the **on_start()** and **on_stop()** event implementations of this state, we are just setting the **self._start_time** value in the case of the former, and printing out some log messages.

In [ ]:
def scan_callback(self, data):
    self.data = data

Finally, in our callback function for the laser subscriber, we are just saving the incoming laser data for later use in the variable of the **sel.data** class.

And that's it! So, at this point, you have just created your first FlexBe state. But as for being able to use this state for something, you'll need to include it into a behavior. So now, we are going to create a new behavior in FlexBe, and we are going to use our new state in it.

## Creating a new behavior

So, let's now try to use our new state in FlexBe! For that, you'll need to first launch the FlexBe application.

<p style="background:#EE9023;color:white;">**Exercise 1.2**</p>

a) To launch the FlexBe application, you can use the following command.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
roslaunch flexbe_app flexbe_full.launch

<span style="color:red;">**IMPORTANT NOTE:** If you launch FlexBe from different Web Shells, remember to always source the workspace before with the below command.</span>

In [ ]:
source ~/catkin_ws/devel/setup.bash

<span style="color:red;">**If you don't source the workspace, you may encounter erros.**</span>

Now, click the icon with a screen in the top-right corner of the IDE window 
<img src="img/font-awesome_desktop.png"/> 
in order to open the Graphic Interface window. In the new window, the FlexBe app should have appeared.

<img src="img/flexbe_app.png" width="800" />

b) In the **Behavior Dashborad** window, you'll see several sections. First of all, let's fill in the **Overview** section. Take a look at the following caption and fill in yours.

<img src="img/flexbe_overview.png" width="400" />

c) Now, let's move to the **Statemachine Editor** window. Here you will have the main Drag & Drop window.

In the top section of this window, you'll see a button that is called **Add State**. When you click it, a new pop-up will appear at the right side of the screen.

<img src="img/flexbe_addstate.png" width="200" />

<img src="img/flexbe_addstate_panel1.png" width="400" />

Here, you can select the package where your new state is located, **turtlebot_flexbe_states**. Once you do that, you will have your state available for selection.

<img src="img/flexbe_addstate_panel2.png" width="400" />

Select your state, give it a name, and click on the Add button. Automatically, a new block will appear in your drag & drop section.

<div style="text-align: left">
<img src="img/flexbe_addsatate_panel3.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/flexbe_dragdrop1.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

d) Now, let's reorganize our blocks a little bit, and click on the **Auto-Connect** button at the top-right corner of the FlexBe App. You will see how, automatically, some connections are created.

<div style="text-align: left">
<img src="img/flexbe_autoconnect.png" width="300" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/flexbe_dragdrop2.png" width="500" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

e) Complete the connections of your state by connecting the **done** outcome to **finished**. It should look something like this:

<img src="img/flexbe_dragdrop3.png" width="700"/>

f) The next step will be to set up the parameters, which are the inputs of your state. Remember that your state has three inputs: **speed**, **travel_dist**, and **obstacle_dist**.

In [ ]:
def __init__(self, speed, travel_dist, obstacle_dist):

Go back to the **Behavior Dashboard** and localize the **Behavior Paramters** section.

<img src="img/flexbe_params2.png" width="400"/>

Let's first add a paramter called **my_speed**. Choose the **Numeric** type, set the default value to 0.4, and set the maximum and minimum values to 0 and 1, respectively. You can have a look at the image below.

<img src="img/flexbe_params3.png" width="400"/>

Do the same for the other two parameters: travel distance and obstacle distance. You can have a look at the images below.

<div style="text-align: left">
<img src="img/flexbe_turn_angle.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
<img src="img/flexbe_turn_speed.png" width="400" style="float: left; margin: 0px 0px 15px 15px;"/>
</div>

Once you have created all three parameters, let's go back to the **Statemachine editor**. Click on the block of your state and you should see a panel like this one in the right side of the window. Here you will need to connect all the inputs of your state to the parameters that you have just created.

<img src="img/flexbe_params5.png" width="400"/>

When you select your parameters, you will see that they are automatically remapped to the variables of the class, in the form of **self.parameter**. In the end, you should have something like this.

<img src="img/flexbe_params6.png" width="400"/>

Finally, remember to click on the **Apply** button to apply the recent changes. Also, click on the **Save Behavior** button. Saving the behavior is VERY IMPORTANT. If you don't, you won't be able to move the final step.

<img src="img/flexbe_save_behavior.png" width="400"/>

g) The last step will be to actually run our behavior. So, first of all, let's move to the **Runtime Control** window. You will see something like this:

<img src="img/flexbe_runtime.png" width="800"/>

Here you will be able to execute your behavior by clicking on the **Start Execution** button. Also, as you can see, you can modify the values of the parameters however you want. So, let's click on the **Start Execution** button and see our new behavior in action!

You should see the Turtlebot robot moving forwards, and stopping once it's too close to the wall.

<img src="img/flexbe1.gif" width="600" />

<span style="color:red;">**¡¡VERY IMPORTANT!!** Remember that, whenever you are done with the Exercise, you can reset the robot's position by hitting on the button **Reset the simulation!**, at the top-right corner of the simulation screen.</span>

<img src="img/reset_sim.png" width="300" />

<p style="background:#EE9023;color:white;">**End of Exercise 1.2**</p>

Great! So at this point, you already know how to create a new FlexBe behavior, and also how to add a new state to it. Now, you could add as many more states as you wish in order to create more complex behaviors. In the following unit, we are going to add a new state to our behavior!

## Congratulations!! You are now capable of creating your own FlexBe behavior, and adding new states to it.